<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Hands-on Lab: Complete the Data Collection API Lab


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


Install the below libraries


In [ ]:
!pip install requests
!pip install pandas
!pip install numpy

## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [1]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [2]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [3]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [4]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [5]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [6]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [7]:
response = requests.get(spacex_url)

Check the content of the response


In [ ]:
print(response.content)

You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [9]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [10]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [16]:
# Use json_normalize meethod to convert the json result into a dataframe
# Convert the JSON data into a pandas DataFrame using json_normalize
# Decode the response content as JSON
data = response.json()
df = pd.json_normalize(data)

Using the dataframe <code>data</code> print the first 5 rows


In [46]:
data['landingPad'].isnull().any()

KeyError: 'landingPad'

In [17]:
# Get the head of the dataframe
df.head()

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]","Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/f9/4a/ZboXReNb_o.png,https://images2.imgbox.com/80/a2/bkWotCIS_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/6c/cb/na1tzhHs_o.png,https://images2.imgbox.com/4a/80/k1oAkY0k_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],"Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the SpaceX Falcon 1",[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': 

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [54]:
import pandas as pd
import datetime

# Assuming 'data' is a list of dictionaries, convert it to a DataFrame
if not isinstance(data, pd.DataFrame):
    data = pd.DataFrame(data)  # Convert the list to a DataFrame if it's not already one

# Check the structure of data
print(type(data))  # Should be <class 'pandas.core.frame.DataFrame'>
print(data.head())  # Check the first few rows of data

# Keep only the desired features
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# Remove rows with multiple cores or payloads
data = data[data['cores'].map(lambda x: isinstance(x, list) and len(x) == 1)]  # Check if 'cores' is a list and length is 1
data = data[data['payloads'].map(lambda x: isinstance(x, list) and len(x) == 1)]  # Check if 'payloads' is a list and length is 1

# Extract the single core and payload details safely
def extract_core(core_list):
    if isinstance(core_list, list) and len(core_list) > 0 and isinstance(core_list[0], dict):
        return core_list[0].get('core', None)
    return None

def extract_payload(payload_list):
    if isinstance(payload_list, list) and len(payload_list) > 0 and isinstance(payload_list[0], dict):
        return payload_list[0].get('payload_id', None)
    return None

data['cores'] = data['cores'].map(extract_core)
data['payloads'] = data['payloads'].map(extract_payload)

# Convert 'date_utc' to datetime and extract the date (removes the time)
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

# Check the result
print(data.head())

<class 'pandas.core.frame.DataFrame'>
       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                      None                    NaN  False     0.0   

                     rocket success  \
0  5e9d0d95eda69955f709d1eb   False   
1  5e9d0d95eda69955f709d1eb   False   
2  5e9d0d95eda69955f709d1eb   False   
3  5e9d0d95eda69955f709d1eb    True   
4  5e9d0d95eda69955f709d1eb    True   

                                                                                                            failures  \
0                                                [{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]   
1            [{'time': 301, 'altitude': 289, 'reason': 'harmonic oscilla

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* **From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.**

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [55]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [56]:
BoosterVersion

[]

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [57]:
# Call getBoosterVersion
getBoosterVersion(data)

the list has now been update 


In [58]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

we can apply the rest of the  functions here:


In [67]:
# Call getLaunchSite
getLaunchSite(data)

In [68]:
# Call getPayloadData
getPayloadData(data)

In [69]:
# Call getCoreData
getCoreData(data)

TypeError: string indices must be integers, not 'str'

In [70]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core_list in data['cores']:
        if isinstance(core_list, dict):  # If it's a dict, wrap it in a list to maintain consistency
            core_list = [core_list]
        if isinstance(core_list, list):  # Ensure it's a list
            for core in core_list:
                if isinstance(core, dict):  # Check if core is a dictionary
                    # Safely access the core data using .get() to avoid key errors
                    Block.append(core.get('block', None))
                    ReusedCount.append(core.get('reuse_count', None))
                    Serial.append(core.get('serial', None))
                    Outcome.append(f"{core.get('landing_success', None)} {core.get('landing_type', None)}")
                    Flights.append(core.get('flight', None))
                    GridFins.append(core.get('gridfins', None))
                    Reused.append(core.get('reused', None))
                    Legs.append(core.get('legs', None))
                    LandingPad.append(core.get('landpad', None))
                else:
                    # Append None if it's not a valid dictionary
                    Block.append(None)
                    ReusedCount.append(None)
                    Serial.append(None)
                    Outcome.append(None)
                    Flights.append(None)
                    GridFins.append(None)
                    Reused.append(None)
                    Legs.append(None)
                    LandingPad.append(None)
        else:
            # If core_list is not a list, append None for all values
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
            Outcome.append(None)
            Flights.append(None)
            GridFins.append(None)
            Reused.append(None)
            Legs.append(None)
            LandingPad.append(None)

# Example call to the function
getCoreData(data)

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [71]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}


Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [72]:
# Create a data from launch_dict
# Create a DataFrame from launch_dict
launch_df = pd.DataFrame(launch_dict)

# Display the first few rows of the DataFrame
print(launch_df.head())

ValueError: All arrays must be of the same length

Show the summary of the dataframe


In [ ]:
# Show the head of the dataframe
launch_df.head()

### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [73]:
# Check the length of each list in the dictionary
for key, value in launch_dict.items():
    print(f"{key}: {len(value)}")  # This will print the length of each list

# Find the maximum length
max_length = max([len(lst) for lst in launch_dict.values()])

# Ensure all lists are of the same length by padding shorter lists with None
for key in launch_dict:
    if len(launch_dict[key]) < max_length:
        # Append None to the list until its length matches the max length
        launch_dict[key] += [None] * (max_length - len(launch_dict[key]))

# Now create the DataFrame
launch_df = pd.DataFrame(launch_dict)

# Display the first few rows to check the result
print(launch_df.head())

FlightNumber: 94
Date: 94
BoosterVersion: 94
PayloadMass: 0
Orbit: 0
LaunchSite: 188
Outcome: 94
Flights: 94
GridFins: 94
Reused: 94
Legs: 94
LandingPad: 94
Block: 94
ReusedCount: 94
Serial: 94
Longitude: 188
Latitude: 188
   FlightNumber        Date BoosterVersion PayloadMass Orbit       LaunchSite  \
0           1.0  2006-03-24       Falcon 1        None  None  Kwajalein Atoll   
1           2.0  2007-03-21       Falcon 1        None  None  Kwajalein Atoll   
2           4.0  2008-09-28       Falcon 1        None  None  Kwajalein Atoll   
3           5.0  2009-07-13       Falcon 1        None  None  Kwajalein Atoll   
4           6.0  2010-06-04       Falcon 9        None  None     CCSFS SLC 40   

  Outcome Flights GridFins Reused  Legs LandingPad Block ReusedCount Serial  \
0    None    None     None   None  None       None  None        None   None   
1    None    None     None   None  None       None  None        None   None   
2    None    None     None   None  None       None  N

In [74]:
# Hint data['BoosterVersion']!='Falcon 1'
# Filter the dataframe to only include Falcon 9 launches
falcon_9_df = launch_df[launch_df['BoosterVersion'].str.contains('Falcon 9', na=False)]

# Display the first few rows of the filtered dataframe
print(falcon_9_df.head())

   FlightNumber        Date BoosterVersion PayloadMass Orbit    LaunchSite  \
4           6.0  2010-06-04       Falcon 9        None  None  CCSFS SLC 40   
5           8.0  2012-05-22       Falcon 9        None  None  CCSFS SLC 40   
6          10.0  2013-03-01       Falcon 9        None  None  CCSFS SLC 40   
7          11.0  2013-09-29       Falcon 9        None  None   VAFB SLC 4E   
8          12.0  2013-12-03       Falcon 9        None  None  CCSFS SLC 40   

  Outcome Flights GridFins Reused  Legs LandingPad Block ReusedCount Serial  \
4    None    None     None   None  None       None  None        None   None   
5    None    None     None   None  None       None  None        None   None   
6    None    None     None   None  None       None  None        None   None   
7    None    None     None   None  None       None  None        None   None   
8    None    None     None   None  None       None  None        None   None   

    Longitude   Latitude  
4  -80.577366  28.561857  
5 

Now that we have removed some values we should reset the FlgihtNumber column


In [76]:
falcon_9_df.loc[:,'FlightNumber'] = list(range(1, falcon_9_df.shape[0]+1))
falcon_9_df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1.0,2010-06-04,Falcon 9,None,None,CCSFS SLC 40,None,None,None,None,None,None,None,None,None,-80.577366,28.561857
5,2.0,2012-05-22,Falcon 9,None,None,CCSFS SLC 40,None,None,None,None,None,None,None,None,None,-80.577366,28.561857
6,3.0,2013-03-01,Falcon 9,None,None,CCSFS SLC 40,None,None,None,None,None,None,None,None,None,-80.577366,28.561857
7,4.0,2013-09-29,Falcon 9,None,None,VAFB SLC 4E,None,None,None,None,None,None,None,None,None,-120.610829,34.632093
8,5.0,2013-12-03,Falcon 9,None,None,CCSFS SLC 40,None,None,None,None,None,None,None,None,None,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,86.0,2020-09-03,Falcon 9,None,None,KSC LC 39A,None,None,None,None,None,None,None,None,None,-80.603956,28.608058
90,87.0,2020-10-06,Falcon 9,None,None,KSC LC 39A,None,None,None,None,None,None,None,None,None,-80.603956,28.608058
91,88.0,2020-10-18,Falcon 9,None,None,KSC LC 39A,None,None,None,None,None,None,None,None,None,-80.603956,28.608058
92,89.0,2020-10-24,Falcon 9,None,None,CCSFS SLC 40,None,None,None,None,None,None,None,None,None,-80.577366,28.561857


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [78]:
falcon_9_df.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass       90
Orbit             90
LaunchSite         0
Outcome           90
Flights           90
GridFins          90
Reused            90
Legs              90
LandingPad        90
Block             90
ReusedCount       90
Serial            90
Longitude          0
Latitude           0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [80]:
# Calculate the mean value of PayloadMass column

# Replace the np.nan values with its mean value

# Ensure PayloadMass column is numeric (just in case some entries are not)
launch_df['PayloadMass'] = pd.to_numeric(launch_df['PayloadMass'], errors='coerce')

# Calculate the mean of the PayloadMass column, ignoring NaN values
mean_payload_mass = launch_df['PayloadMass'].mean()

# Print the mean value
print(f"Mean PayloadMass: {mean_payload_mass}")

# Replace NaN values in the PayloadMass column with the mean value
launch_df['PayloadMass'].fillna(mean_payload_mass, inplace=True)

# Check the updated PayloadMass column
print(launch_df['PayloadMass'].head())

Mean PayloadMass: nan
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: PayloadMass, dtype: float64


/var/folders/7k/2mbzj1vx1t7csf6mftq94ydm0000gn/T/ipykernel_52719/2445349667.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  launch_df['PayloadMass'].fillna(mean_payload_mass, inplace=True)


In [81]:
print(falcon9_data.columns)


Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings', 'date'],
      dtype='object')


In [49]:
falcon9_data


,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings,date


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


In [83]:
falcon_9_df.to_csv('dataset_part_1.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


<!--## Change Log
-->


<!--

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|
-->


Copyright © 2021 IBM Corporation. All rights reserved.
